In [ ]:
from dwcal import delay_weighted_cal as dwcal
import numpy as np
import pyuvdata
from pyuvdata import utils
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# Get data
data, model = dwcal.get_test_data(
    model_path="dwcal/data/model.uvfits",
    data_path="dwcal/data/data.uvfits",
    use_autos=False,
)

In [ ]:
# Plot visibility error

# Difference visibilities
data.filename = [""]
model.filename = [""]
diff_vis = data.diff_vis(model, inplace=False)

# FFT across frequency
delay_array = np.fft.fftfreq(diff_vis.Nfreqs, d=diff_vis.channel_width)
delay_array = np.fft.fftshift(delay_array)
fft_abs = np.abs(np.fft.fftshift(np.fft.fft(diff_vis.data_array, axis=2), axes=2))
fft_abs *= diff_vis.channel_width

# Average in baseline length bins
nbins = 300
bl_lengths = np.sqrt(np.sum(diff_vis.uvw_array**2.0, axis=1))
bl_bin_edges = np.linspace(np.min(bl_lengths), np.max(bl_lengths), num=nbins + 1)
binned_variance = np.full([nbins, diff_vis.Nfreqs], np.nan, dtype="float")
for bin_ind in range(nbins):
    bl_inds = np.where(
        (bl_lengths > bl_bin_edges[bin_ind])
        & (bl_lengths <= bl_bin_edges[bin_ind + 1])
    )[0]
    if len(bl_inds) > 0:
        binned_variance[bin_ind, :] = np.mean(
            fft_abs[bl_inds, 0, :, 0] ** 2.0, axis=0
        )

# Plot
use_cmap = matplotlib.cm.get_cmap("inferno").copy()
use_cmap.set_bad(color="whitesmoke")
vmin = 1e9
vmax = 1e13
norm = matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax)
plt.imshow(
    binned_variance.T,
    origin="lower",
    interpolation="none",
    cmap=use_cmap,
    norm=norm,
    extent=[
        np.min(bl_bin_edges),
        np.max(bl_bin_edges),
        np.min(delay_array) * 1e6,
        np.max(delay_array) * 1e6,
    ],
    aspect="auto",
)

# Add horizon lines
plt.plot(
    [np.min(bl_bin_edges), np.max(bl_bin_edges)],
    [
        np.min(bl_bin_edges) / 3e8 * 1e6,
        np.max(bl_bin_edges) / 3e8 * 1e6,
    ],
    "--",
    color="white",
    linewidth=1.0,
)
plt.plot(
    [np.min(bl_bin_edges), np.max(bl_bin_edges)],
    [
        -np.min(bl_bin_edges) / 3e8 * 1e6,
        -np.max(bl_bin_edges) / 3e8 * 1e6,
    ],
    "--",
    color="white",
    linewidth=1.0,
)

cbar = plt.colorbar(extend="both")
cbar.ax.set_ylabel(
    "Visibility Variance (Jy$^{2}$/s$^2$)", rotation=270, labelpad=15
)
plt.xlabel("Baseline Length (m)")
plt.ylim([np.min(delay_array) * 1e6, np.max(delay_array) * 1e6])
plt.ylabel("Delay ($\mu$s)")
plt.show()

In [ ]:
# Create randomized gains
random_gains_stddev = 0.01
random_gains = np.random.normal(
    1.0,
    random_gains_stddev,
    size=(data.Nants_data, data.Nfreqs),
) + 1.0j * np.random.normal(
    0.0,
    random_gains_stddev,
    size=(data.Nants_data, data.Nfreqs),
)

# Ensure that the phase of the gains is mean-zero for each frequency
avg_angle = np.arctan2(
    np.mean(np.sin(np.angle(random_gains)), axis=0),
    np.mean(np.cos(np.angle(random_gains)), axis=0),
)
random_gains *= np.cos(avg_angle) - 1j * np.sin(avg_angle)

# Create UVCal object
antenna_list = np.unique([data.ant_1_array, data.ant_2_array])
random_gains_cal = dwcal.initialize_cal(data, antenna_list, gains=random_gains)
random_gains_cal.gain_convention = "divide"  # Apply initial calibration as division

# Apply gains to data
pyuvdata.utils.uvcalibrate(data, random_gains_cal, inplace=True, time_check=False)

In [ ]:
cal_diagonal = dwcal.calibration_optimization(
    data,
    model,
    weight_mat_option="diagonal",
)

In [ ]:
cal_dwcal = dwcal.calibration_optimization(
    data,
    model,
    weight_mat_option="exponential window fit",
)

In [ ]:
# Plot results

def histogram_plot_2d(
    plot_data,
    plot_range=0.005,
    nbins=50,
    colorbar_range=None,
    savepath=None,
    title="",
    axis_label="Gain Error",
    center_on_one=False,
):

    bins = np.linspace(-plot_range, plot_range, num=nbins + 1)
    hist, x_edges, y_edges = np.histogram2d(
        np.real(plot_data).flatten(),
        np.imag(plot_data).flatten(),
        bins=[bins_real, bins_imag],
    )

    if colorbar_range is None:
        colorbar_range = [0, np.max(hist)]

    plt.figure(figsize=[7, 5.5])
    plt.imshow(
        hist.T,
        interpolation="none",
        origin="lower",
        extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]],
        aspect="equal",
        cmap="inferno",
        vmin=colorbar_range[0],
        vmax=colorbar_range[1],
    )
    cbar = plt.colorbar(extend="max")
    cbar.ax.set_ylabel("Histogram Count", rotation=270, labelpad=20)
    plt.xlabel("{}, Real Part".format(axis_label))
    plt.ylabel("{}, Imaginary Part".format(axis_label))
    plt.title(title)
    plt.tight_layout()
    plt.grid(linewidth=0.2, color="white")
    plt.show()

cal_diag_error = diff_gains(cal_diagonal, random_gains_cal)
histogram_plot_2d(
    cal_diag_error.gain_array.flatten(),
    colorbar_range=[0,350],
    title="Sky-Based Calibration",
)

cal_dwcal_error = diff_gains(cal_dwcal, random_gains_cal)
histogram_plot_2d(
    cal_dwcal_error.gain_array.flatten(),
    colorbar_range=[0,350],
    title="DWCal",
)